In [1]:
#!pip install Flask Pillow flask-cors

In [ ]:
import sys
import os
import time
import uuid
import json
import base64
import requests
import io # 바이트 스트림 처리를 위해 import

from flask import Flask, request, jsonify
from flask_cors import CORS
from PIL import Image # Pillow(PIL) import
from colorthief import ColorThief # ColorThief import

# Google Gemini API 라이브러리 임포트
import google.generativeai as genai

# --- 1. Google Gemini API 설정 ---
CLOVA_API_URL = 'clova_api_url'
CLOVA_SECRET_KEY = 'clova_api_key'
genai_key = 'genai_key'
try:
    genai.configure(api_key=genai_key) # 본인의 API 키 입력
except AttributeError as e:
    print("Gemini API 키 설정에 실패했습니다. API 키를 확인하세요.")
    print(f"오류: {e}")
    sys.exit(1)

# 사용할 Gemini 모델 설정 (gemini-2.5-flash-preview)
gemini_model = genai.GenerativeModel('gemini-2.5-flash-preview-09-2025') 

# JSON 응답을 위한 GenerationConfig 설정
gemini_json_config = genai.GenerationConfig(
    response_mime_type="application/json"
)

# Gemini 호출을 위한 시스템 프롬프트 (JSON 스키마 포함)
gemini_system_prompt = """
당신은 텍스트 목록을 받아 음식 이름만 식별하는 한국어 메뉴판 분석 전문가입니다.
다음은 텍스트 목록입니다. 
1.  이 텍스트가 '음식', '음료', 또는 '주류'의 이름이라고 판단되면, `isFood`를 `true`로 설정합니다.
2.  음식, 음료, 주류 이름이 아니라면(예: 가게 이름, 주소, 가격, 설명 문구, 수량 등), `isFood`를 `false`로 설정합니다.
3.  `isFood`가 `true`인 경우에만 다음 작업을 수행합니다:
    a.  `originalText` (원본 텍스트)를 바탕으로 `translatedText` (영어로 번역)를 생성합니다.
    translatedText에 들어갈 내용은 한국어의 로마자표기법이 아닌 실제와 같은 발음을 먼저 표시하고 소괄호 속에 의미를 번역한 내용이 들어가야합니다.
    b.  `description` (음식에 대한 50자 내외의 간략한 영어 설명과,
    해당 식품 섭취시 알러지 위험성 정보, 비건푸드 해당사항 정보, 할랄푸드 해당사항정보가 반드시 포함되어있어야 하며, 없으면 없다고 표시되어야 함)을 생성합니다.
4.  `isFood`가 `false`인 경우, `translatedText`와 `description`은 빈 문자열("")로 둡니다.

반드시 다음 JSON 스키마 형식에 맞춰 응답해야 합니다.

{
  "type": "object",
  "properties": {
    "items": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "originalText": { "type": "string" },
          "isFood": { "type": "boolean" },
          "translatedText": { "type": "string" },
          "description": { "type": "string" }
        },
        "required": ["originalText", "isFood", "translatedText", "description"]
      }
    }
  },
  "required": ["items"]
}
"""


# --- 2. Flask 앱 설정 ---
app = Flask(__name__)
CORS(app) 

# --- 3. Naver Clova OCR 설정 ---
CLOVA_API_URL = 'https://asvkrrh9wm.apigw.ntruss.com/custom/v1/46882/e7d892787a42dc4981929aa36512de1188aa9a15b9d653d4223d255d57fc577c/general'
CLOVA_SECRET_KEY = 'WU9tZVJKdmdPeExLbG5ic2RpVVRIcnFITE96aWdSSmM='

# --- 4. Gemini 호출 함수 ---
def analyze_texts_with_gemini(text_list):
    """
    텍스트 목록을 Gemini API로 보내 음식 정보를 분석하고 JSON 결과를 반환합니다.
    """
    if not text_list:
        return []

    try:
        # 사용자 프롬프트 생성 (텍스트 목록을 단순 문자열로 결합)
        user_prompt = "다음 텍스트 목록을 분석해주세요:\n" + "\n".join(text_list)
        
        # Gemini API 호출
        response = gemini_model.generate_content(
            user_prompt + "\n" + gemini_system_prompt,
            generation_config=gemini_json_config
        )
        
        # 응답 파싱
        result_json = json.loads(response.text)
        return result_json.get("items", [])

    except Exception as e:
        print(f"Gemini API 호출 오류: {e}")
        # 오류 발생 시 빈 리스트 반환
        return []

# --- 5. Flask 라우트 (메인 OCR 처리) ---
@app.route('/ocr', methods=['POST'])
def ocr_process():
    try:
        # 1. 클라이언트로부터 Base64 이미지 수신
        data = request.json
        image_data_url = data['image'] # "data:image/png;base64,iVBORw0KGgo..."
        
        header, encoded = image_data_url.split(",", 1)
        image_bytes = base64.b64decode(encoded)

        # ColorThief를 위한 PIL 이미지 객체 생성
        try:
            original_image = Image.open(io.BytesIO(image_bytes))
        except Exception as e:
            print(f"Pillow 이미지 열기 실패: {e}")
            original_image = None # 실패 시 None으로 설정

        # 2. Clova OCR API 요청 준비
        request_json = {
            'images': [
                {
                    'format': 'png',
                    'name': 'capture'
                }
            ],
            'requestId': str(uuid.uuid4()),
            'version': 'V2',
            'timestamp': int(round(time.time() * 1000))
        }
        payload = {'message': json.dumps(request_json).encode('UTF-8')}
        files = [
            ('file', image_bytes) # 원본 바이트를 그대로 전송
        ]
        headers = {
            'X-OCR-SECRET': CLOVA_SECRET_KEY
        }

        # 3. Clova OCR API 호출 (15초 타임아웃 추가)
        response = requests.request("POST", CLOVA_API_URL, headers=headers, data=payload, files=files, timeout=15)
        
        response.raise_for_status() 
        
        ocr_result = response.json() 

        print("--- 1. CLOVA API 응답 원본 ---")
        print(json.dumps(ocr_result, indent=2, ensure_ascii=False))
        print("-------------------------------")

        # 4. OCR 결과에서 텍스트 목록 추출
        clova_fields = []
        text_list_for_gemini = []
        if ocr_result.get('images'):
            clova_fields = ocr_result['images'][0].get('fields', [])
            for field in clova_fields:
                raw_text = field.get('inferText', '')
                text_list_for_gemini.append(raw_text) # 원본 텍스트(띄어쓰기 포함)를 전송

        if not clova_fields:
            print("Clova OCR 결과가 비어있습니다. (fields 없음)")
            return jsonify(ocr_result) 

        # 5. Gemini API로 텍스트 목록 분석
        print(f"--- 2. Gemini 분석 요청 (텍스트 {len(text_list_for_gemini)}개) ---")
        gemini_results = analyze_texts_with_gemini(text_list_for_gemini) 
        print(f"--- 3. Gemini 분석 결과 (항목 {len(gemini_results)}개) ---")
        print(json.dumps(gemini_results, indent=2, ensure_ascii=False))
        print("-----------------------------------")

        # 6. Gemini 결과를 딕셔너리(Map)로 변환 (빠른 조회용)
        gemini_map = {item['originalText']: item for item in gemini_results}

        # 7. Clova 결과(fields)와 Gemini 맵(gemini_map)을 텍스트 기준으로 병합
        final_fields = [] # 음식이 아닌 항목을 제외한 최종 리스트

        for field in clova_fields:
            infer_text = field.get('inferText', '')
            gemini_data = gemini_map.get(infer_text) # 텍스트가 정확히 일치하는 항목을 찾음
            
            if not gemini_data or not gemini_data.get('isFood', False):
                continue 

            # --- 이 항목은 음식이므로(isFood: true) 처리 계속 ---
            
            field['isFood'] = True
            field['translatedText'] = gemini_data.get('translatedText', '')
            field['description'] = gemini_data.get('description', '')
            
            # (4) 색상 추출 (음식 항목에 대해서만 수행)
            if original_image:
                try:
                    vertices = field['boundingPoly']['vertices']
                    xCoords = [v['x'] for v in vertices]
                    yCoords = [v['y'] for v in vertices]
                    
                    box = (min(xCoords), min(yCoords), max(xCoords), max(yCoords))
                    cropped_image = original_image.crop(box)

                    img_byte_arr = io.BytesIO()
                    cropped_image.save(img_byte_arr, format='PNG')
                    img_byte_arr.seek(0) 

                    color_thief = ColorThief(img_byte_arr)
                    palette = color_thief.get_palette(color_count=2, quality=10) 
                    
                    bg_color_rgb = palette[0]
                    text_color_rgb = palette[1]

                    bg_color_css = f"rgb({bg_color_rgb[0]}, {bg_color_rgb[1]}, {bg_color_rgb[2]})"
                    text_color_css = f"rgb({text_color_rgb[0]}, {text_color_rgb[1]}, {text_color_rgb[2]})"

                    field['detectedBackgroundColor'] = bg_color_css
                    field['detectedTextColor'] = text_color_css
                except Exception as e:
                    print(f"색상 추출 실패 (Field: {field.get('inferText')}): {e}")
                    # [수정됨] 실패 시 불투명 흰색 배경 반환
                    field['detectedBackgroundColor'] = "rgb(255, 255, 255)"
                    field['detectedTextColor'] = "rgb(0, 0, 0)"
            
            final_fields.append(field)

        # 8. 최종 결과(ocr_result)의 fields를 필터링된 final_fields로 교체
        ocr_result['images'][0]['fields'] = final_fields
        
        print(f"--- 4. 최종 전송 (음식 {len(final_fields)}개) ---")

        # 9. 수정된 OCR 결과(음식만 포함)를 클라이언트로 반환
        return jsonify(ocr_result)

    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP 오류 발생: {http_err}")
        print(f"응답 내용: {http_err.response.text}")
        return jsonify({"error": f"Clova API 오류: {http_err.response.status_code}", "message": http_err.response.text}), 500
    except Exception as e:
        print(f"전체 오류 발생: {e}")
        return jsonify({"error": str(e)}), 500

# --- 6. Flask 서버 실행 ---
if __name__ == '__main__':
    app.run(host='192.168.219.51', port=5000)

